查询pip下载当前所使用的源

In [1]:
pip config list

global.index-url='https://mirror.sjtu.edu.cn/pypi/web/simple/'
Note: you may need to restart the kernel to use updated packages.


##### 换源

常使用的源：<br />
清华：https://pypi.tuna.tsinghua.edu.cn/simple/ <br />
阿里云：http://mirrors.aliyun.com/pypi/simple/ <br />
中国科技大学：https://pypi.mirrors.ustc.edu.cn/simple/ <br />
华中科技大学：http://pypi.hustunique.com/simple/ <br />
上海交通大学：https://mirror.sjtu.edu.cn/pypi/web/simple/ <br />
豆瓣：http://pypi.douban.com/simple/ <br />

In [ ]:
pip config set global.index-url 源地址

##### 查看nii文件

In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt

# 读取.nii文件
nii_path = r'/home/aistudio/─╘PET═╝╧ё╖╓╬Ў║═╝▓▓б╘д▓т╠Ї╒╜╚№╣л┐к╩¤╛▌/Test/1.nii'
nii_img = nib.load(nii_path)

# 获取图像数据
nii_data = nii_img.get_fdata()

# 可选：查看图像属性
print("图像形状:", nii_data.shape)
print("像素尺寸:", nii_img.header.get_zooms())

# 可根据需要更改切片索引
slice_index = nii_data.shape[2]  // 2
plt.subplot(2,2,1)
plt.imshow(nii_data[:, :, slice_index, 0], cmap='gray') 
plt.subplot(2,2,2)
plt.imshow(nii_data[:, slice_index,:, 0], cmap='gray') 
plt.subplot(2,2,3)
plt.imshow(nii_data[slice_index ,: , : , 0], cmap='gray') 